# Block-based scheduler for truck arrival smoothing 
Author Ilias Parmaksizoglou

This notebook provided a quick tour on how to use the Block-based scheduler for truck arrival smoothing tool (3.1). THe needed input is :
* Matrix of incoming trucks with stated flexibilities and preferences for each truck belonging to a company 
* Events table for the port, associating the arrival of a truck with an event on the port, e.g., arrival of a vessel

The output of this tool is needed by further tools and contains a coarse schedule of truck arrivals across time windows

In [95]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import os

# Load Data

The two inputs are stored in the core folder, in the files "info_trucks.csv" and "info_events.csv". Some further parameters are derived from the nature of truck arrivals, while some port parameters are set accoding to user's preference. Specifically the following parameters can be set as desired:

* pn : penalty factor for assignment above capacity
* cap_w : max capacity inside the time-window 
* cap_s : max capacity inside the time-window for single destination

In [85]:
core_dir = os.path.join(os.getcwd(),"core")
trucks_namefile = "info_trucks.csv"
trucks_eventsfile = "info_events.csv"

# Load Trucks and Companies and Events
trucks_data = pd.read_csv(os.path.join(core_dir,trucks_namefile))
events_data = pd.read_csv(os.path.join(core_dir,trucks_eventsfile))
all_companies = trucks_data["Company"].unique()

# Initialize MILP Variables
multidict_input = {}
for i,row in trucks_data.iterrows():
    multidict_input[row[0]] = row[1], row[2],row[3],row[4],row[5],row[6],row[7]
trucks,company,F_t_m,P_t,F_t_p,V_t,TY_t,AS_w = gp.multidict(multidict_input)

for i,row in events_data.iterrows():
    multidict_input[row[0]] = row[1], row[2]
jobs, start, end=  gp.multidict(multidict_input)

no_windows = len(set(trucks_data['Flex Early']))
windows = range(1,no_windows+1)
no_destinations = len(set(trucks_data['Destination']))
destinations = range(1,no_destinations+1)
no_trucks =  len((trucks_data['Destination']))

# User stated Parameters
pn = 25 # Penalty for assignment outside flexibility
cap_w = 9
cap_s = 5 # Capacity for a single yard 

In [86]:

# Create optimization model
model = gp.Model('tas-tcs')

# Derived Variables
z = model.addVars(trucks, windows,jobs,vtype=GRB.BINARY, name="z")
d = model.addVars(trucks,vtype=GRB.CONTINUOUS, name="d")
s = model.addVars(windows,trucks,vtype=GRB.BINARY, name="s")  


Constraint 2 guarantees that each truck $t$ is assigned to a time-window $w$ consistent with the event $j_t$. This requirement can lead to violations of the flexibility zone that will ultimately lead to an auction (as the selected $w$ could be smaller than  $F_t^-$ or larger than $F_t^+$), but this approach fully embraces our flexibility framework. Constraint 3 prevents the assignment of a truck to a time-window not consistent with its associated event, while constraint 4 prevents the assignment of a truck to an event which is not $j_t$.

In [87]:
# Constraints (2) - (4)
for t in trucks:
    for j in jobs:
        if j == V_t[t]:
            model.addConstr(sum(z[t, w ,j] for w in windows if w >=start[j] and w <=end[j]) == 1)
            model.addConstr(sum(z[t, w ,j] for w in windows if w <start[j] or w >end[j]) == 0)
        else:
            model.addConstr(sum(z[t, w, j] for w in windows) == 0)

 Constraints 5-6 compute the deviation from preferred arrival time-window, as they model the absolute value $|w-P_t|$ for every truck. The two constraints provide a lower bound for the deviation for the cases where assignment is before or after the preferred window. In case that assignment is within the flexibility zone, the deviation will be equal to the lower bound since this is a minimization problem.

In [88]:
# Constraint (5) - (6)
for t in trucks:
    for w in windows:
        for j in jobs:
            if j == V_t[t]:
                model.addConstr(w-P_t[t]-d[t] <= no_windows*(1-z[t,w,j]))
                model.addConstr(-w+P_t[t]-d[t] <= no_windows*(1-z[t,w,j]))      

Constraints 7-8 guarantee that when the assignment is outside the flexibility zone of a truck, deviation is equal to $Pn$. In particular, the two constraints are dummy when $z_{twj}$=0, while they force $d_t$=$Pn$ when $z_{twj}$=1.
Constraint 9 link an assignment outside the flexibility zone of a truck with the auction indicator for the truck's preferred arrival time-window. Essentially, when a truck cannot be assigned to a time-window within its flexibility zone, this constraint triggers an auction for its preferred time-window. Note that $P_t$ can change dynamically for those trucks that are requested to specify a new preferred time-window.

In [89]:
# Constraint (7) - (9)
for t in trucks:
    for w in windows:
        for j in jobs:
            if j == V_t[t] and w not in range(F_t_m[t],F_t_p[t]+1):
                model.addConstr(d[t]-pn <= 1-z[t,w,j])
                model.addConstr(d[t] >= pn*z[t,w,j])      
                model.addConstr(s[P_t[t],t] >= z[t,w,j])


Constraints 10-11 are capacity constraints formulated, respectively, for the total number of trucks allowed in the port at a single time-window, and for the total number of accepted trucks accessing a specific destination $y$ at time-window $w$. They are both needed as one regulates capacity at the whole terminal area level, the other one at the single destination level.

In [91]:
# Constraint (10)
for w in windows:
    model.addConstr(sum(z[t, w, j] for t in trucks for j in jobs) <=cap_w)

# Constraint (11)
for w in windows:
    for y in destinations:
        model.addConstr(sum(z[t, w, j] for t in trucks for j in jobs if TY_t[t] == y) <=cap_s)

Constraints 12 are initially an empty set, as they are added on-the-fly if needed due to an auction mechanism. Constraint set 13 ensures that trucks are serviced at their procured time-window, while constraint set~\ref{cam:15} prevents new auctions to occur for a time-window that has already been resolved.

In [92]:
# Constraint (12)
for t in trucks:
    for w in windows:
        for j in jobs:
            if j == V_t[t] and w == AS_w[t]:
                model.addConstr(z[t,w,j] == 1)

# Constraint (13)
for t in trucks:
    if AS_w[t] > 0 :
        model.addConstr(sum(s[AS_w[t],t_prime] for t_prime in trucks)==0)




The Objective function minimizes the total sum of deviations from the preferred arrival time-window across all trucks. The deviation of each truck is obtained in two different ways. It is equal to the absolute difference between the assigned and preferred time-window when the assignment is within the flexibility zone of the truck, or it is equal to $Pn$ otherwise. In practice, $Pn$ should be set to be relatively higher than any ``flexible" deviation to disincentivize assignment outside the flexibility zone. If $Pn$ is relatively low, the algorithm will prioritize minimization of deviations, even if that may lead to an auction. Conversely, if $Pn$ is relatively high, the avoidance of auctions will be the primary objective.

In [93]:
# Solve The Model
# Objective Function
model.setObjective(sum(d[t] for t in trucks), GRB.MINIMIZE)
model.setParam("LogToConsole",0)
model.optimize()

48.0


If the results is a feasilbe assignment then the schedule is save inside the core folders. In case of an assignment that results to an auction, a display message infrorms the user that the auction mechanism must be triggered

In [ ]:
# Get the Results
if model.Status == GRB.OPTIMAL:
    auc_trucks = []
    for w in windows:
        for t in trucks:
            if s[w,t].x >=0.99999:
                if d[t].x == pn:
                    auction_window = w
                    auc_trucks.append(t)
                    for t in trucks:
                        for j in jobs:
                            if z[t,auction_window,j].x >=0.99:
                                auc_trucks.append(t)
    if auc_trucks == []:
        block_schedule = pd.DataFrame(columns=["Truck","Window","Destination"])
        count = 0
        for w in windows:
            for t in trucks:
                for j in jobs:
                    if z[t,w,j].x == 1:
                        list_truck = [t,w,TY_t[t]]
                        block_schedule.loc[count] = list_truck
                        count+=1
        block_schedule = block_schedule.set_index("Truck")
        block_schedule.to_csv(os.path.join(core_dir,"schedule.csv"))
    else:
        print(f"Auction is needed in time window {auction_window} for Trucks: {auc_trucks} ")
    